As the dataset is unlabelled, I needed to develop training data set for learning based classification by my own but with the least manual involvement. I used the lexicon-based classifier, VADER, to annotate the training data for the learning-based classifier, it is expected to maintain the efficiency of lexicon-based approach while harvesting high-performance of learning-based approach. I also expanded existing general purpose sentiment polarity lexica by adding health-relating sentiment polarity lexica by identifying domain-specific words which appear often. Each term was marked into positive and negative manually, aligning with the domain context. Then, I considered the dataset with over or below certain threshold of the score as polar dataset. Furthermore, in order to avoid biasness of the training set, I expanded the training set by labeling manually around 5-10% amount of dataset randomly chosen from each divided interval, which is not considered as polar or neutral data. 

In [7]:
from vaderSentiment import vaderSentiment
import json 

In [10]:
senti = vaderSentiment.SentimentIntensityAnalyzer()

### Add our own words with own score 

In [25]:
with open("healthcareDictionary.json") as f:
    dictionary =json.loads(f.read())

print("Number of added Negative Words: ", len(dictionary.get("newNegWord")))
print("Number of added Positive Words: ", len(dictionary.get("newPosWord")))
print("Number of added Ngram Words: ", len(dictionary.get("newNgram")))

senti.lexicon.update(dictionary.get("newNegWord"))
senti.lexicon.update(dictionary.get("newPosWord"))
senti.lexicon.update(dictionary.get("newNgram"))

Number of added Negative Words:  258
Number of added Positive Words:  151
Number of added Ngram Words:  158


In [8]:
Ngram = [i.replace("_", " ") for i in newNgram.keys()]

## simple test case

In [157]:
senti.polarity_scores("this is not side_effect")

{'neg': 0.0, 'neu': 0.547, 'pos': 0.453, 'compound': 0.357}

# further data processing 

- I changed emoticon into character, which follows format like :"smiling_face":.
- In order to catch the emoticon on Vader, I removed ":" and "_" for tokenization.  
- change bigram words into with underbar between the word


In [10]:
import pandas as pd
import re 

In [13]:
cdf = pd.read_csv("cDMARD without URL Vader classifier ver9.csv")

In [14]:
bdf = pd.read_csv("bDMARD without URL Vader classifier ver9.csv")

In [16]:
Ngrams = list(word.replace("_", " ") for word in dictionary.get("newNgram").keys())

ngrams = "|".join(Ngrams)

for i in range(len(cdf)):
    try:
        c = cdf.tweet[i].lower()
        if re.search(ngrams, temp):
            
            for word in re.findall(ngrams, temp):
                temp = re.sub(word, word.replace(" ", "_"), temp)
                
        if re.search("\:{0,1}[a-z_-]{1,}\:", temp):
            for emo in re.findall("\:{0,1}[a-z_-]{1,}\:", temp):
                temp = temp.replace(emo, re.sub("\:|\_", " ", emo))
        cdf.tweet[i] = temp
           
    except Exception:
        print("error", c.tweet[i])

gold is also used in infrared shielding..coloredglass...certain gold salts,still used4 anti-inflammatories in medicine.?yellows from?! smiling face with smiling eyes  grinning face with sweat  squinting face with tongue  smiling face ️
certain gold salts are still used as anti-inflammatories in medicine.
#goldfacts certain gold salts are still used as anti-inflammatories in medicine.
some gold salts used still as anti-inflammatories in medicine.
“: some arthritis medications contain gold salts, which is used as an anti-inflammatory” fancy stuff here  new moon_face  ok hand  dizzy 


C:\Users\jennyj\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [114]:
scores = []
for tweet in cdf.tweet:
    score = senti.polarity_scores(tweet)
    scores.append(score)

In [119]:
cDF_score = pd.DataFrame({"index":cdf.index, "id":cdf.id, "user_id": cdf.user_id, "tweet": cdf.tweet, "score": scores})
cDF_score['polarity']  = [score['compound'] for score in scores]

cDF_score.to_csv("cDMARD without URL Vader classifier ver9.csv")

In [18]:
scores1 = []
for i, tweet in enumerate(bdf.tweet):
    try:
        score = senti.polarity_scores(tweet)
        scores1.append(score)
    except Exception:
        print(i)
        print(tweet)
        scores1.append({'compound': None})

In [19]:
bDF_score = pd.DataFrame({"index":bdf.index, "id":bdf.id, "user_id": bdf.user_id, "tweet": bdf.tweet, "score": scores1})

bDF_score['polarity']  = [score['compound'] for score in scores1] 

In [20]:
bDF_score.to_csv("bDAMRD without URL Vader classifier.csv")

In [28]:
senti.polarity_scores("first time out of the house properly in almost 3 weeks since the start of my relapse . things are moving up :smiling face  infliximab crohnsdisease")

{'neg': 0.094, 'neu': 0.719, 'pos': 0.188, 'compound': 0.4588}